In [4]:
import parselmouth
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import pandas as pd
import subprocess
import time


In [5]:
# the differnt emotions in the files
emotions = ['anxiety', 'boredom', 'cold-anger', 'contempt', 'despair', 'disgust', 'elation', 'happy',
            'hot-anger', 'interest', 'neutral', 'panic',    'pride',    'sadness', 'shame']
clr =      ["red",  "cyan",   "orange",  "purple",   "magenta", "green",    "yellow",  "pink",
            "yellow", "green", "magenta", "purple",  "orange", "cyan",      "red"]

speakers = ['mf_001', 'cc_001', 'jg_001', 'mm_001', 'cl_001', 'gg_001', 'mk_001']

In [6]:
# directory to opensmile
smile = "/opensmile-2.3.0/inst/bin"
cwd = os.getcwd()
toFiles = cwd + "/speech_files/"


run = './SMILExtract'
conf = '../../config/IS09_emotion.conf'
output = "myfeatureset.csv"

In [ ]:
# saving the names of the files
names = []
# change directory to opensmile
os.chdir(smile)
# batching processing the audio file
# all the files should be contained in the folder called speech_files
for wave_file in glob.glob(toFiles + "*.wav"):
    
    name = wave_file.split("/")[-1][:len(wave_file.split("/")[-1])-4]
    print("Processing {}...".format(name))
#     print(run , '-C' , conf ,  '-I' , wave_file, '-csvoutput', output)
    p = subprocess.Popen([run , '-C' , conf ,  '-I' , wave_file, '-csvoutput', output])
    names.append(name)
    p.wait()

# reading the final feature file    
data = pd.read_csv(output, sep=';')

#changing names of the file
print("number of files processed and featuresized: ", len(names))
for x in range(len(names)):
    data['name'][x] = names[x]

# change back directory
os.chdir(cwd)

#saving file
data.to_csv(output, index=False)

In [7]:
data = pd.read_csv(output)

In [ ]:
reports = {'p':[], 'nb':[], 'r':[], 'c':[], 's':[]}

In [8]:
# one speaker out cross validation
# get the data for test and train
for spk in speakers:
    
    test = [[],[]]
    train = [[],[]]
    
    for sp in speakers:
        if sp == spk:
            test[0] = test[0] + sdata[sp][0]
            test[1] = test[1] + sdata[sp][1]
        else:
            train[0] = train[0] + sdata[sp][0]
            train[1] = train[1] + sdata[sp][1]

    print('test speaker: ', spk)

    testX = np.array(test[1])
    trainX = np.array(train[1])
    

    trainY = []
    testY = []

    for x in range(len(train[0])):
        e = train[0][x]
        trainY.append(emotions.index(e))

    for x in range(len(test[0])):
        e = test[0][x]
        testY.append(emotions.index(e))

    trainY = np.array(trainY)
    testY = np.array(testY)


    

    #multilayer perceptron
    mlf = MLPClassifier(hidden_layer_sizes=(250,500,150, 100), activation='tanh',
                        max_iter=500, alpha=0.0001,learning_rate="adaptive",
                        learning_rate_init=0.0001,solver='sgd', verbose=False,  
                        random_state=110,tol=0.000000000001, nesterovs_momentum=True, warm_start=True)
   
    mlf.fit(trainX, trainY)
    y_pred = mlf.predict(testX)
    print('perceptron', spk)
    rep = classification_report(testY, y_pred, target_names=emotions)
    print(rep)
    reports['p'].append(rep)
    
    
#     # SVM
#     clf = LinearSVC(random_state=0, tol=1e-5, max_iter=2000)
#     clf.fit(trainX, trainY)
#     y_pred = clf.predict(testX)
#     print(classification_report(testY, y_pred, target_names=emotions))
  
    
    #Ridge Regression
    rlf = RidgeClassifier( max_iter=100000).fit(trainX, trainY)
    y_pred = rlf.predict(testX)
    print('Ridge', spk)
    rep = classification_report(testY, y_pred, target_names=emotions)
    print(rep)
    reports['r'].append(rep)
    
    # Naive Bayes Gaussian
    glf = GaussianNB().fit(trainX, trainY)
    y_pred = glf.predict(testX)
    print('naive bayes', spk)
    rep = classification_report(testY, y_pred, target_names=emotions)
    print(rep)
    reports['nb'].append(rep)
    
    
    # nearest centroid
    cnlf = NearestCentroid().fit(trainX, trainY)
    y_pred = cnlf.predict(testX)
    print('centroid', spk)
    rep = classification_report(testY, y_pred, target_names=emotions)
    print(rep)
    reports['c'].append(rep)
    
#     break

NameError: name 'sdata' is not defined